In [1]:
import numpy as np
import pandas as pd
from scipy import stats, integrate
import matplotlib.pyplot as plt
import re
import seaborn as sns
sns.set(color_codes=True)

import tabula


In [2]:
df_pdf = tabula.read_pdf('kcp-mag-2021 v2.pdf', 
    pages= 'all',
    lattice=True,
    pandas_options={"header": [0, 1, 2, 3]},
    #area=[0, 0, 50, 100],
    relative_area=True,
    #multiple_tables=False
                        )

df = df_pdf[0]
for df_temp in df_pdf[1:]:
    df = df.append(df_temp,ignore_index=True)

Got stderr: дек 15, 2021 11:44:08 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
дек 15, 2021 11:44:08 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
дек 15, 2021 11:44:10 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>



In [3]:
df

,0,1,2,3,4,5,6,7,8,9
0,Код,Направление подготовки\rНаправленность (профил...,Дополнительная информация,Контрольные цифры приема,Грант\r(за счет средств\rАкадемии),По договорам об оказании платных образовательн...,NaN,NaN,NaN,NaN
1,NaN,NaN,Общие условия,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,очное,заочное,очно-\rзаочное,очное,заочное,очно-заочное,NaN,NaN
3,1,2,3,5,6,7,NaN,8,9,10
4,УСЛОВИЯ ПОСТУПЛЕНИЯ:\rФедеральное государствен...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
163,42.04.05,Медиакоммуникации\r- по программе магистратуры...,NaN,0,0,0,0,0,0,NaN
164,42.04.02,Журналистика\r- по программе магистратуры в пр...,NaN,0,0,0,0,Цифры приема будут\rразмещены на сайте\rАкадем...,0,NaN
165,46.04.01,История\r- по программе магистратуры в предела...,На образовательной программе\rвыделено 5 мест ...,0,0,0,5,0,0,NaN
166,46.04.01,История\r- по программе магистратуры в предела...,На образовательной программе\rвыделено 3 места...,0,0,0,3,0,0,NaN


In [4]:
df_mos = df[ df[1].str.contains('по программе|по совокупности', na=False) ].reset_index()
df_mos.tail()

print('дубликаты: {0}' .format( df_mos[[0,1]].duplicated().sum() ))

df_mos = df_mos[[0,1]].copy().drop_duplicates()
#df_mos

дубликаты: 0


In [28]:
df_one = df_mos[df_mos[1].str.contains('по программе магистратуры')]

def program_name(patern, tmp):
    tmp = tmp.replace('\r'," ").split(':',1)[1]
    #tmp = tmp.replace('  '," ")
    tmp = tmp.replace(re.findall(patern, tmp )[-1],'')
    " ".join(tmp.split())
    tmp = tmp.strip()
    return tmp

def facult_name(patern, tmp):
    tmp = tmp.replace('\r'," ")
    tmp = re.findall(patern, tmp )[-1].strip()
    return tmp

df_one_mos = pd.DataFrame([])
df_one_mos['код'] = df_one.iloc[:, 0]
df_one_mos['стандарт'] = df_one.iloc[:,1].map(lambda x: x.split('\r',1)[0])
df_one_mos['программа'] =  df_one.iloc[:,1].map(lambda x: program_name("\([ а-яa-zА-ЯA-Z0-9-: ]*\)", x))
df_one_mos['подразделение'] =  df_one.iloc[:,1].map(lambda x: facult_name("\(([ а-яa-zА-ЯA-Z0-9-: ]*)\)", x))
df_one_mos['программа'] = df_one_mos['программа'].map(lambda x: re.sub('[ё]', 'е', x))




' Россия-Европа: политические и геополитические исследования (российско-французская программа) (ИГСУ)'

['российско-французская программа', 'ИГСУ']

In [24]:
df_one.iloc[120,1]

'Зарубежное регионоведение\r- по программе магистратуры в пределах направления\rподготовки:\rРоссия-Европа: политические и геополитические\rисследования (российско-французская программа)\r(ИГСУ)'

In [5]:
df_multi = df_mos[df_mos[1].str.contains('по совокупности')]

def multi_transform(tmp):
    tmp_3 = tmp.iloc[1].split(':\r',1)[1]
    tmp_3 = " ".join(tmp_3.split())
    tmp_3 = re.split('\d.',tmp_3)[1:]
    tmp_3 = np.char.replace(tmp_3,'\r',' ')
    tmp_3 = np.char.replace(tmp_3,'.','')
    dic = {1: tmp.iloc[0],
           2: tmp.iloc[1].split('\r',1)[0],
           3: tmp_3
          }
    df_tmp = pd.DataFrame(dic)
    df_tmp[4] = df_tmp[3].map(lambda x: facult_name("\(([ а-яa-zА-ЯA-Z0-9-: ]*)\)", x))
    df_tmp[3] = df_tmp[3].map(lambda x: x.replace(re.findall("\([ а-яa-zА-ЯA-Z0-9-: ]*\)", x )[-1],'').strip() )
    return df_tmp

tmp = [ multi_transform(df_multi.loc[idx]) for idx in df_multi.index.values]

df_multi_mos = pd.concat(tmp, ignore_index=True)
df_multi_mos.columns = df_one_mos.columns.values
#display(df_multi_mos.head(3))
#display(df_multi_mos.tail(3))


In [6]:
df_moscow = pd.concat([df_one_mos, df_multi_mos], ignore_index=True)
df_moscow.drop_duplicates(inplace=True)
df_moscow.head(3)

,код,стандарт,программа,подразделение
0,09.04.03,Прикладная информатика,Цифровые технологии в экономике,ФИТАД ЭМИТ
1,09.04.03,Прикладная информатика,Анализ данных и искусственный интеллект/ Data ...,ФИТАД ЭМИТ
2,37.04.01,Психология,Психология управления,ИОН


In [32]:
df_pdf = tabula.read_pdf('kcp-mag-filialu-2021 v2.pdf', 
    pages= 'all',
    lattice=True,
    pandas_options={"header": [0, 1, 2, 3]},
    #area=[0, 0, 50, 100],
    relative_area=True,
    #multiple_tables=False
                        )

df = df_pdf[0]
for df_temp in df_pdf[1:]:
    df = df.append(df_temp,ignore_index=True)
df.head(10)

searchfor = ['Направление','Общие',"очное"]
mask = df[1].str.contains('|'.join(searchfor), na=False)|df[2].str.contains('|'.join(searchfor), na=False)
df = df[~mask][[0,1]].copy()


mask_uslovie = df[0].str.contains("УСЛОВИЯ", na=False)


#df[0] = df[0].str.replace('УСЛОВИЯ ПОСТУПЛЕНИЯ:\r','')
df[0] = df[0].str.replace('УСЛОВИЯ ПОСТУПЛЕНИЯ:\r','')
df[0] = df[0].map(lambda x: x.split('– программы')[0]) # внимание тут разное тире в разных строках!!! 
df[0] = df[0].map(lambda x: x.split('- программы')[0]) # внимание тут разное тире в разных строках!!! 
df[0] = df[0].map(lambda x: x.split('- филиал')[0])
mask_list = np.array(df[mask_uslovie].index)


with pd.option_context('display.max_rows', None):
         display(df)

mask_list

,0,1
3,Алтайский филиал,NaN
4,07.04.04,Градостроительство\rпо программе магистратуры ...
5,37.04.01,Психология\rпо программе магистратуры в рамках...
6,38.04.01,Экономика\rпо программе магистратуры в рамках ...
7,38.04.04,Государственное и муниципальное управление\rпо...
8,38.04.08,Финансы и кредит\rпо совокупности программ маг...
9,40.04.01,Юриспруденция\rпо совокупности программ магист...
10,Брянский филиал,NaN
11,38.04.03,Управление персоналом\rпо программе магистрату...
12,38.04.04,Государственное и муниципальное управление\rпо...


array([  3,  10,  14,  21,  31,  33,  40,  43,  49,  51,  53,  56,  61,
        66,  68,  80,  82,  91, 104, 107, 114, 116, 128, 133, 135])

In [34]:
def func_dic(df_test):
    dic = {} 
    code = df_test[0]
    dic.update({0: code})
    tmp = df_test[1]
    napravlenie = tmp.split('\r',1)[0]
    dic.update({1: napravlenie})
    tmp = tmp.split('\r',2)[2]
    tmp = " ".join(tmp.split())
    tmp = re.split('\d.',tmp)[1:]
    tmp = np.char.replace(tmp,'\r',' ')
    tmp = np.char.replace(tmp,'.','')
    tmp = np.char.strip(tmp)
    dic.update({2: tmp})
    df_tmp = pd.DataFrame(dic)
    return df_tmp


df_multi_institute = []
mask_shape = mask_list.shape[0]
for i in range(mask_shape):
    if i < mask_shape-1:
        end = mask_list[i+1]-1
    else:
        end = df.index.max()
    
    df_fil = df.loc[mask_list[i]:end]
    
    tmp = [ func_dic(df_fil.loc[idx]) for idx in df_fil.iloc[1:].index.values]    
    df_multi = pd.concat(tmp, ignore_index=True)    
    df_multi[3] = df_fil.iloc[0][0].strip()

    
    df_multi = df_multi.drop_duplicates()
    df_multi_institute.append(df_multi)

df_multi_institute = pd.concat(df_multi_institute, ignore_index=True) 
df_multi_institute.columns = df_one_mos.columns.values

df_multi_institute['программа'] = df_multi_institute['программа'].str.replace('ё','е')

df_multi_institute.head(3)


,код,стандарт,программа,подразделение
0,07.04.04,Градостроительство,Современная урбанистика,Алтайский филиал
1,37.04.01,Психология,Прикладная социальная психология,Алтайский филиал
2,38.04.01,Экономика,Экономика здравоохранения,Алтайский филиал


In [40]:
mask_list.shape[0]

25

# Перечень всех программ магистратур Академии

In [9]:
magistratura = pd.concat([df_moscow,df_multi_institute],ignore_index=True)

with pd.option_context('display.max_rows', None):
         display(magistratura)

,код,стандарт,программа,подразделение
0,09.04.03,Прикладная информатика,Цифровые технологии в экономике,ФИТАД ЭМИТ
1,09.04.03,Прикладная информатика,Анализ данных и искусственный интеллект/ Data ...,ФИТАД ЭМИТ
2,37.04.01,Психология,Психология управления,ИОН
3,37.04.01,Психология,Психология кризисных состояний и клиническая п...,ИОН
4,38.04.01,Экономика,Экономика и финансы,ЭФ ЭМИТ
5,38.04.01,Экономика,Поведенческая экономика,ЭФ ЭМИТ
6,38.04.01,Экономика,Системы больших данных в экономике,ЭФ ЭМИТ
7,38.04.01,Экономика,Цифровая экономика,ЭФ ЭМИТ
8,38.04.01,Экономика,Корпоративная экономика,ЭФ ЭМИТ
9,38.04.01,Экономика,Экономика и право,ЭФ ЭМИТ


In [10]:
tmp = magistratura[magistratura['код']=='38.04.01']

with pd.option_context('display.max_colwidth', None):
         display(tmp)

,код,стандарт,программа,подразделение
4,38.04.01,Экономика,Экономика и финансы,ЭФ ЭМИТ
5,38.04.01,Экономика,Поведенческая экономика,ЭФ ЭМИТ
6,38.04.01,Экономика,Системы больших данных в экономике,ЭФ ЭМИТ
7,38.04.01,Экономика,Цифровая экономика,ЭФ ЭМИТ
8,38.04.01,Экономика,Корпоративная экономика,ЭФ ЭМИТ
9,38.04.01,Экономика,Экономика и право,ЭФ ЭМИТ
10,38.04.01,Экономика,Экономика медиаиндустрии,ЭФ ЭМИТ
11,38.04.01,Экономика,Экономика и контрольно-надзорная деятельность,ЭФ ЭМИТ
12,38.04.01,Экономика,"Банки, финансы, инвестиции",ФФБ
13,38.04.01,Экономика,Бухгалтер-аналитик: корпоративная отчетность и финансовое моделирование,ФФБ


In [11]:
tmp = magistratura[magistratura['код']=='38.04.08']

with pd.option_context('display.max_colwidth', None):
         display(tmp)

,код,стандарт,программа,подразделение
96,38.04.08,Финансы и кредит,Прикладные корпоративные финансы,ВШФМ
97,38.04.08,Финансы и кредит,Денежно-кредитное и финансовое регулирование экономики,ФФБ
98,38.04.08,Финансы и кредит,Финансовый инженер,ФФБ
99,38.04.08,Финансы и кредит,Регулирование финансовых рынков и институтов,ИГСУ
100,38.04.08,Финансы и кредит,Финансы и инвестиции,ИФУР
162,38.04.08,Финансы и кредит,Государственные и муниципальные финансы,Алтайский филиал
163,38.04.08,Финансы и кредит,Финансовая диагностика и организационные технологии в бизнесе,Алтайский филиал
179,38.04.08,Финансы и кредит,Финансовый менеджмент,Волгоградский институт управления
188,38.04.08,Финансы и кредит,Финансовая экономика,Дальневосточный институт управления
215,38.04.08,Финансы и кредит,Корпоративные финансы,Нижегородский институт управления


# Перечень всех дубликатов программ магистратур Академии

In [12]:
tmp = magistratura[magistratura['программа'].duplicated(keep=False)].sort_values(by=['программа','код'])

with pd.option_context('display.max_colwidth', None):
         display(tmp)

,код,стандарт,программа,подразделение
95,38.04.05,Бизнес-информатика,Бизнес-аналитика,ФИТАД ЭМИТ
242,38.04.05,Бизнес-информатика,Бизнес-аналитика,Северо-Западный институт управления
72,38.04.04,Государственное и муниципальное управление,Государственная служба и кадровая политика,ИГСУ
219,38.04.04,Государственное и муниципальное управление,Государственная служба и кадровая политика,Петропавловский филиал
250,38.04.04,Государственное и муниципальное управление,Государственная служба и кадровая политика,Северо-Кавказский институт управления
108,40.04.01,Юриспруденция,"Гражданское право, семейное право, международное частное право",ВШП ИГСУ
180,40.04.01,Юриспруденция,"Гражданское право, семейное право, международное частное право",Волгоградский институт управления
30,38.04.02,Менеджмент,Деловое администрирование,ИБДА
206,38.04.02,Менеджмент,Деловое администрирование,Московский областной филиал
44,38.04.02,Менеджмент,Корпоративное управление и стратегическое развитие бизнеса,ВШКУ


# Перечень всех программ магистратур Академии по ЭКОНОМИКЕ

In [13]:
tmp = magistratura[magistratura['код']=='38.04.01'].sort_values(by='программа')

with pd.option_context('display.max_colwidth', None):
         display(tmp)

,код,стандарт,программа,подразделение
220,38.04.01,Экономика,Анализ и моделирование экономичяеских процессов региона,Поволжский институт управления
12,38.04.01,Экономика,"Банки, финансы, инвестиции",ФФБ
13,38.04.01,Экономика,Бухгалтер-аналитик: корпоративная отчетность и финансовое моделирование,ФФБ
20,38.04.01,Экономика,Государственное регулирование экономики,ИГСУ
8,38.04.01,Экономика,Корпоративная экономика,ЭФ ЭМИТ
276,38.04.01,Экономика,Мировая экономика,Южно-Российский институт управления
275,38.04.01,Экономика,Налогообложение и налоговое администрирование,Южно-Российский институт управления
5,38.04.01,Экономика,Поведенческая экономика,ЭФ ЭМИТ
258,38.04.01,Экономика,Региональная экономика,Среднерусский институт управления
6,38.04.01,Экономика,Системы больших данных в экономике,ЭФ ЭМИТ


# Перечень всех программ магистратур Академии по ФИНАНСАМ и КРЕДИТУ

In [14]:
tmp = magistratura[magistratura['код']=='38.04.08'].sort_values(by='программа')

with pd.option_context('display.max_colwidth', None):
         display(tmp)

,код,стандарт,программа,подразделение
162,38.04.08,Финансы и кредит,Государственные и муниципальные финансы,Алтайский филиал
97,38.04.08,Финансы и кредит,Денежно-кредитное и финансовое регулирование экономики,ФФБ
215,38.04.08,Финансы и кредит,Корпоративные финансы,Нижегородский институт управления
96,38.04.08,Финансы и кредит,Прикладные корпоративные финансы,ВШФМ
99,38.04.08,Финансы и кредит,Регулирование финансовых рынков и институтов,ИГСУ
163,38.04.08,Финансы и кредит,Финансовая диагностика и организационные технологии в бизнесе,Алтайский филиал
188,38.04.08,Финансы и кредит,Финансовая экономика,Дальневосточный институт управления
98,38.04.08,Финансы и кредит,Финансовый инженер,ФФБ
179,38.04.08,Финансы и кредит,Финансовый менеджмент,Волгоградский институт управления
262,38.04.08,Финансы и кредит,Финансовый менеджмент,Среднерусский институт управления


In [15]:
# magistratura.to_excel("mag_2021.xlsx")